In [5]:
import string
import requests
import Crypto

In [6]:
hex_digits = "0123456789abcdef"
values = string.ascii_uppercase + string.ascii_lowercase + string.digits + "+/"


def split_chunks(l, n):
    ch = []
    for x in l:
        ch.append(x)
        if len(ch) == n:
            yield ch
            ch = []
    if ch:
        yield ch


def hex_decode(text):
    lookup = {y: x for x, y in enumerate(hex_digits)}

    res = []

    for first, second in split_chunks(text, 2):
        first = lookup[first]
        second = lookup[second]

        byte = first << 4 | second
        res.append(byte)

    return bytes(res)


def groups(data, n):
    group = []
    for x in data:
        group.append(x)
        if len(group) == n:
            yield "".join(group)
            group = []
    if group:
        yield "".join(group)


def b64_groups(data):
    digits = binary_digits(data)
    for num in groups(digits, 6):
        num = int(num, 2)
        yield values[num]


def base64_encode(data):
    res = ""
    for b64 in groups(b64_groups(data), 4):
        res += b64
        for _ in range(4 - len(b64)):
            res += "="

    return res


def b64_to_binary(text):
    for c in text:
        x = values.find(c)
        if x != -1:
            for i in range(6)[::-1]:
                bit = (x >> i) & 1
                yield str(int(bit))


def binary_digits(data):
    for byte in data:
        for i in range(8)[::-1]:
            i = (byte >> i) & 1
            yield str(int(i))


def base64_decode(text):
    res = []
    for gr in groups(b64_to_binary(text), 8):
        if len(gr) != 8:
            break
        res.append(int(gr, 2))

    return bytes(res)


def single_xor(ciphertext, key):
    plain = [x ^ key for x in ciphertext]
    return bytes(plain)


def repeating_xor(data, key):
    res = []
    for i, c in enumerate(data):
        k = key[i % len(key)]
        res.append(c ^ k)
    return bytes(res)

In [7]:
URL = "https://cryptopals.com/static/challenge-data/7.txt"
ciphertext = requests.get(URL).text
ciphertext = base64_decode(ciphertext)

In [8]:
from Crypto.Cipher import AES


def aes_ecb_encrypt(key, block):
    aes = AES.new(key, AES.MODE_ECB)
    return aes.encrypt(block)


def aes_ecb_decrypt(key, block):
    aes = AES.new(key, AES.MODE_ECB)
    return aes.decrypt(block)


for i in range(0, len(ciphertext), 16):
    chunk = ciphertext[i : i + 16]
    chunk = aes_ecb_decrypt(b"YELLOW SUBMARINE", chunk)
    chunk = chunk.decode("ascii")
    print(chunk, end="")
    if i > 150:
        break

I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and